<a href="https://colab.research.google.com/github/albanda/CE888/blob/master/lab4-recommender/rec_latent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jokes Rating

Load required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

Now load the data

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/PauMarquez99/DataScience-CE888/lab6/lab6/jester-data-1.csv", header=None)

In [5]:
data.shape

(24983, 101)

In [17]:
# Replace value for NAN as it does not represent a rating

data = data.replace(99.00, np.NaN)

In [19]:
# Drop column, it is only the number of jokes rated by the user
df = data.iloc[: , 1:]
df.shape

(24983, 100)

In [8]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [9]:
latent_item_features

array([[0.202, 0.039],
       [0.398, 0.696],
       [0.128, 0.444],
       [0.15 , 0.367],
       [0.739, 0.575],
       [0.978, 0.067],
       [0.914, 0.432],
       [0.566, 0.393],
       [0.744, 0.845],
       [0.994, 0.56 ],
       [0.599, 0.185],
       [0.612, 0.9  ],
       [0.783, 0.597],
       [0.843, 0.539],
       [0.369, 0.802],
       [0.27 , 0.685],
       [0.452, 0.437],
       [0.531, 0.961],
       [0.102, 0.311],
       [0.92 , 0.224],
       [0.012, 0.717],
       [0.488, 0.583],
       [0.114, 0.032],
       [0.919, 0.62 ],
       [0.749, 0.693],
       [0.51 , 0.694],
       [0.88 , 0.373],
       [0.455, 0.343],
       [0.269, 0.447],
       [0.675, 0.643],
       [0.002, 0.87 ],
       [0.714, 0.647],
       [0.819, 0.21 ],
       [0.502, 0.935],
       [0.868, 0.056],
       [0.539, 0.055],
       [0.16 , 0.039],
       [0.013, 0.356],
       [0.799, 0.189],
       [0.77 , 0.438],
       [0.089, 0.134],
       [0.424, 0.623],
       [0.586, 0.722],
       [0.2

In [10]:
latent_user_preferences

array([[0.797, 0.751],
       [0.998, 0.456],
       [0.103, 0.741],
       ...,
       [0.714, 0.033],
       [0.657, 0.635],
       [0.415, 0.578]])

In [11]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [12]:
num_iter = 300000
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10000), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/300000:	MSE=815.715246


KeyboardInterrupt: 

In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

In [ ]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction